In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [ ]:
pip install fuzzywuzzy

In [ ]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
import pandas as pd
import csv
import string

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
podes = pd.read_excel('/content/gdrive/MyDrive/PROJECT MAGANG BPS 2021/GOOGLE MAPS X PODES/new matching running/Data/KOTA DENPASAR.xlsx')
podes_num = pd.read_excel("/content/gdrive/MyDrive/PROJECT MAGANG BPS 2021/GOOGLE MAPS X PODES/new matching running/Data/KOTA DENPASAR_num.xlsx")
podes_non_num = pd.read_excel("/content/gdrive/MyDrive/PROJECT MAGANG BPS 2021/GOOGLE MAPS X PODES/new matching running/Data/KOTA DENPASAR_non_num.xlsx")
gmaps = pd.read_csv('/content/gdrive/MyDrive/PROJECT MAGANG BPS 2021/GOOGLE MAPS X PODES/new matching running/Data/Data Infrastruktur Clean.csv')
gmaps = gmaps.drop_duplicates(subset = None, keep = 'first', inplace= False)

In [ ]:
##preprocessing nama infra pendidikan
#read slang word, read aja, tapi jadinya pake dict2 yang langsung define, bukan dict1
f = open('/content/gdrive/MyDrive/PROJECT MAGANG BPS 2021/GOOGLE MAPS X PODES/slang.txt', 'r')
dict1 = {}
for line in f:
  k, v = line.strip().split(':')
  dict1[k.strip()] = v.strip()
f.close()

In [ ]:
##lowering the string
gmaps['nama_infrastruktur'] = [entry.lower() for entry in gmaps['nama_infrastruktur']]
podes['nama'] = [entry.lower() for entry in podes['nama']]

In [ ]:
##remove punctuation atau tanda baca
def remove_punctuation(text):
    return text.translate(str.maketrans("", "", string.punctuation))

gmaps['nama_infrastruktur'] = gmaps['nama_infrastruktur'].apply(remove_punctuation)
podes['nama'] = podes['nama'].apply(remove_punctuation)

In [ ]:
#try chunk
address = "123 north anywhere SD N street"

#lowering each string in dataframe


dict2={"sd negeri":"sd",
       "sd n ":"sd ",
       "sd no":"sd",
       "sd negri":"sd",
       "sdn":"sd",
       "sdnegri":"sd",
       "sdnegeri":"sd",
       "smp no":"smp",
       "smp negeri":"smp",
       "smp n ":"smp ",
       "smp negri":"smp",
       "smpn":"smp",
       "smpnegri":"smp",
       "smpnegeri":"smp",
       "sekolah dasar no":"sd",
       "sekolah dasar negeri no":"sd",
       "sekolah dasar negri no":"sd",
       "sekolah dasar negeri":"sd",
       "sekolah dasar negri":"sd",
       "sekolah menengah pertama no":"smp",
       "sekolah menengah pertama negeri no":"smp",
       "sekolah menengah pertama negri no":"smp",
       "sekolah menengah pertama negeri":"smp",
       "sekolah menengah pertama negri":"smp"
       }
dict3 ={"north":"N",
       "SD N":"SD Negeri",
        "SD Negri":"SD Negeri"}

for word, initial in dict2.items():
    address1 = address.lower()
    address = address1.replace(word.lower(), initial)

print (address)

123 north anywhere sd street


In [ ]:
#AN EXAMPLE
#apply normalize term function
sstring = 'SD N'
for word, initial in dict2.items():
  sstring1 = sstring.lower()
  sstring = sstring1.replace(word.lower(), initial)
print(sstring)

sd n


In [ ]:
#string normalization on nama and nama_infrastruktur columns in podes and gmamps dataframe
for i in range(0, len(podes['nama'])):
  for word, initial in dict2.items():
    sstring1 = podes.iloc[i,1]
    podes.iloc[i,1] = sstring1.replace(word.lower(), initial)

for i in range(0, len(gmaps['nama_infrastruktur'])):
  for word, initial in dict2.items():
    sstring2 = gmaps.iloc[i,6]
    gmaps.iloc[i,6] = sstring2.replace(word.lower(), initial)


In [ ]:
def match_names(name, list_names, min_score=0):
    max_score = -1
    max_name = ''
    for x in list_names:
        score = fuzz.ratio(name, x)
        if (score > min_score) & (score > max_score):
            max_name = x
            max_score = score
    return (max_name, max_score)

In [ ]:
list_ed_infra = ['SD Negeri', 'SMP Negeri']
if(a.iloc[0,7] in list_ed_infra):
  print('true')
else:
  print('false')

true


In [ ]:
for i in list_ed_infra:
  print(i)

['SD Negeri']
['SMP Negeri']


In [ ]:
a.iloc[0,7]

'SD Negeri'

In [ ]:
a = podes.loc[podes['nama'] == 'sd 17 pemecutan']

In [ ]:
a

,id,nama,deskripsi,latitude,longitude,accuracy,kode_kategori,nama_infra,iddesa_20201,KODEPROV,KODEKAB
260,17499213,sd 17 pemecutan,sebelah kantor dinas kebersihan,-8.661442,115.205552,3.0,30010005,SD Negeri,5171030007,51,71


In [ ]:
def create_dict(podes_temp, gmaps_temp):
    names = []
    list_ed_infra = ['SD Negeri', 'SMP Negeri']
    min_score = 80
    df_to_check = list(podes_temp.nama.unique())
    df_to_compare = list(gmaps_temp.nama_infrastruktur.unique())
    for x in df_to_check:
      a = podes_temp.loc[podes_temp['nama'] == x]
      if(a.iloc[0,7] in list_ed_infra):
        min_score = 98
        print('true')
      else:
        min_score = 80
        print('false') 
      print(a.nama_infra)
      match = match_names(x, df_to_compare, 0)
      print(match)
      print(min_score)
      if match[1] >= min_score:
        name = ('(' + str(x), str(match[0]) + ')')
        names.append(name)
    name_dict = dict(names)
    return name_dict


In [ ]:
##apply normalize term function
def normalized_term(document):
    print(document)
    return [dict1[term] if term in dict1 else term for term in document]

In [ ]:
podes.nama_infra.unique()

array(['SD Negeri', 'Pasar dengan', 'Rumah sakit', 'SMP Negeri',
       'Puskesmas ra', 'Puskesmas ta'], dtype=object)

In [ ]:
df_temp = pd.DataFrame()


for i in podes['iddesa_20201']:
  podes_temp = podes.loc[podes['iddesa_20201'] == i]
  gmaps_temp = gmaps.loc[gmaps['iddesa_asli'] == i]
  
  #podes_names_temp = list(podes_temp.nama.unique())
  #gmaps_names_temp = list(gmaps.nama_infrastruktur.unique())
  infra = create_dict(podes_temp, gmaps)
  data_items = infra.items()
  data_list = list(data_items) 

  data_df = pd.DataFrame(data_list)
  s_df = pd.DataFrame(list([i]*len(data_list)))
  col_concat = pd.concat([s_df, data_df], axis = 1)

  df_temp = pd.concat([df_temp, col_concat])

    

Streaming output truncated to the last 5000 lines.
192    SD Negeri
Name: nama_infra, dtype: object
('sd 5 sanur', 100)
98
true
139    SD Negeri
Name: nama_infra, dtype: object
('sd 16 kesiman', 100)
98
true
140    SD Negeri
Name: nama_infra, dtype: object
('sd 12 kesiman', 100)
98
false
148    Rumah sakit
Name: nama_infra, dtype: object
('rumah sakit umum dharma yadnya', 70)
80
true
213    SD Negeri
Name: nama_infra, dtype: object
('sd 10 kesiman', 100)
98
true
231    SD Negeri
Name: nama_infra, dtype: object
('sd 7 kesiman', 100)
98
true
232    SD Negeri
Name: nama_infra, dtype: object
('sd 17 kesiman', 100)
98
false
233    Pasar dengan
Name: nama_infra, dtype: object
('pasar kertha', 100)
80
true
139    SD Negeri
Name: nama_infra, dtype: object
('sd 16 kesiman', 100)
98
true
140    SD Negeri
Name: nama_infra, dtype: object
('sd 12 kesiman', 100)
98
false
148    Rumah sakit
Name: nama_infra, dtype: object
('rumah sakit umum dharma yadnya', 70)
80
true
213    SD Negeri
Name: nama_infr

In [ ]:
df_temp = pd.DataFrame()


for i in podes['iddesa_20201']:
  podes_temp = podes.loc[podes['iddesa_20201'] == i]
  gmaps_temp = gmaps.loc[gmaps['iddesa_asli'] == i]
  
  podes_names_temp = list(podes_temp.nama.unique())
  gmaps_names_temp = list(gmaps.nama_infrastruktur.unique())
  infra = create_dict(podes_names_temp, gmaps_names_temp)
  data_items = infra.items()
  data_list = list(data_items) 

  data_df = pd.DataFrame(data_list)
  s_df = pd.DataFrame(list([i]*len(data_list)))
  col_concat = pd.concat([s_df, data_df], axis = 1)

  df_temp = pd.concat([df_temp, col_concat])

    

('sd 9 peguyangan', 100)
('pasar kangkung', 88)
('smp wisata sanur', 73)
('sd 7 peguyangan', 100)
('sd 2 peguyangan', 93)
('pasar agung peninjoan', 75)
('sd 9 peguyangan', 100)
('pasar kangkung', 88)
('smp wisata sanur', 73)
('sd 7 peguyangan', 100)
('sd 2 peguyangan', 93)
('pasar agung peninjoan', 75)
('rumah sakit umum daerah wangaya', 100)
('sd 4 dauh puri', 100)
('sd 20 dangin puri', 100)
('sd 9 dauh puri', 100)
('rs mmc', 50)
('sd 33 dangin puri', 100)
('sd 21 dauh puri', 93)
('sd 17 dauh puri', 100)
('sd 5 peguyangan', 100)
('sd 10 sumerta', 57)
('tk darma praja', 80)
('warung prama', 72)
('tk pembina denpasar utara', 72)
('pasar swadaya', 71)
('rs thelabel', 63)
('pasar badung', 78)
('sd 1 sesetan', 100)
('pasar rakyat sesetan', 89)
('pasar tradisional pasar sumuh', 81)
('sd 2 sesetan', 100)
('smp 6 denpasar', 100)
('smp 6 denpasar', 71)
('sd 7 sesetan', 100)
('sd 11 sesetan', 100)
('sd 18 sesetan', 100)
('sd 3 sesetan', 100)
('sd 13 sesetan', 100)
('sd 6 sesetan', 100)
('pasar 

In [ ]:
df_n_dup = df_temp.drop_duplicates(subset = None, keep = 'first', inplace= False)

In [ ]:
podes.head()

,id,nama,deskripsi,latitude,longitude,accuracy,kode_kategori,nama_infra,iddesa_20201,KODEPROV,KODEKAB
0,2174145,sd 9 peguyangan,sd no 9 peguyangan,-8.622226,115.229518,4.000000,30010005,SD Negeri,5171031011,51,71
1,1386186,pasar agung,pintu masuk pasar agung,-8.622813,115.230217,4.000000,30030002,Pasar dengan,5171031011,51,71
2,6192790,rumah sakit umum daerah wangaya,20001,-8.648401,115.213391,9.648001,30020001,Rumah sakit,5171031002,51,71
3,19656801,rs hewan,fakultas hewan,-8.691732,115.216393,13.653000,30020001,Rumah sakit,5171010003,51,71
4,19657604,pasar gaduh,pasar senggol,-8.690296,115.217583,4.551000,30030003,Pasar dengan,5171010003,51,71


In [ ]:
df_temp.head()

,0,0,1
0,5171031011,(sd 9 peguyangan,sd 9 peguyangan)
1,5171031011,(sd 7 peguyangan,sd 7 peguyangan)
0,5171031011,(sd 9 peguyangan,sd 9 peguyangan)
1,5171031011,(sd 7 peguyangan,sd 7 peguyangan)
0,5171031002,(rumah sakit umum daerah wangaya,rumah sakit umum daerah wangaya)


In [ ]:
df_n_dup.to_csv('df_final_no_dup_new7.csv')

In [ ]:
df_temp.to_csv('data.csv')

In [ ]:
a = ['221710059']
rep = a*3

In [ ]:
rep

['221710059', '221710059', '221710059']

In [ ]:
podes_names = list(podes.nama.unique())
podes_num_names = list(podes_num.nama.unique())
podes_non_num_names = list(podes_non_num.nama.unique())
gmaps_names = list(gmaps.nama_infrastruktur.unique())

In [ ]:
infra = create_dict(podes_names, gmaps_names, 80)
infra_non_edu = create_dict(podes_non_num_names, gmaps_names, 80)
infra_edu = create_dict(podes_num_names, gmaps_names, 95)

('Sd Negeri 9 Peguyangan', 70)
('Pasar Kangkung', 80)
('Rumah Sakit Umum Daerah Wangaya', 100)
('Pasar Begawan', 57)
('Pasar Badung', 70)
('Sekolah Dasar Negeri 1 Pedungan', 84)
('SD Negeri 3 Ubung', 67)
('SD Negeri 4 Ubung', 74)
('SD Negeri 4 Ubung', 67)
('SD Negeri 6 Panjer', 100)
('Sd Negeri 1 Sesetan', 81)
('Pasar Rakyat Sesetan', 79)
('Bali Royal Hospital', 64)
('Sekolah Dasar Negeri 1 Sumerta', 97)
('SMP PGRI 4 Denpasar', 100)
('TK PELITA DESA', 47)
('Surya Husadha Hospital', 56)
('UGD Prima Medika Hospital', 91)
('SD Negeri 3 Renon', 83)
('SDN 29 Dangin Puri', 89)
('SD Negeri 2 Dangin Puri', 98)
('SD 13 SESETAN', 71)
('Pasar Kumbasari', 100)
('SD Negeri 19 Dauh Puri', 77)
('Pasar tradisional pasar sumuh', 77)
('Rumah Sakit Umum Pusat Sanglah', 78)
('Lab PA RSU Surya Husadha', 68)
('Pasar Pedungan II', 72)
('smp 3 denpasar', 58)
('SD Negeri 1 Sanur', 100)
('Pasar Intaran', 85)
('Koperasi Pasar Abiantimbul', 73)
('Sekolah Dasar No. 8 Pemecutan', 95)
('SD Negeri 1 Renon', 50)
('Pas

In [ ]:
df = pd.DataFrame()
data_dict = {"a": 1, "b": 2, "c": 3}
data_dict1 = {"d": 4, "e": 5, "f": 6}
data_items = data_dict.items()
data_list = list(data_items)
df.append(data_list)



,0,1
0,a,1
1,b,2
2,c,3


In [ ]:
df.append(data_list1)

,0,1
0,d,4
1,e,5
2,f,6


In [ ]:
df

,col 1,col 2
0,a,1
1,b,2
2,c,3


In [ ]:
##the result with by the column
with open("Hasil/infra_non_pendidikan.csv", "w", newline="") as f:
    w = csv.DictWriter(f, infra_non_pendidikan.keys())
    w.writeheader()
    w.writerow(infra_non_pendidikan)

In [ ]:
(pd.DataFrame.from_dict(data= infra_non_edu, orient='index')
   .to_csv('Hasil/infra_non_edu.csv', header=False))

In [ ]:
file = open('try.csv', 'w', newline = '')
gmaps_n = zip(gmaps_names)
with file:
    write = csv.writer(file)
    for gmap in gmaps_n:
        write.writerow(gmap)

In [ ]:
file = open('try_rdp.csv', 'w', newline = '')
gmaps_n = zip(gmaps_rdp_names)
with file:
    write = csv.writer(file)
    for gmap in gmaps_n:
        write.writerow(gmap)

In [ ]:
reader = csv.reader(open('dict_file_non_num.csv', 'r'))
d = {}
for row in reader:
    k, v = row
    d[k] = v

In [ ]:
def read_csv_to_dict(df_name):
    reader = csv.reader(open(df_name, 'r'))
    d = {}
    for row in reader:
        k,v = row
        d[k] = v
    return d

In [ ]:
infra_all = read_csv_to_dict('Hasil/infra_all.csv')

In [ ]:
infra_all = pd.read_csv('Hasil/infra_all.csv', names = ['podes', 'gmaps'])

In [ ]:
infra_all.head()
infra_all.shape

(56, 2)

In [ ]:
infra_all_podes = pd.DataFrame(infra_all['podes'])



In [ ]:
infra_all_podes = pd.DataFrame(infra_all['podes'])
infra_all_gmaps = pd.DataFrame(infra_all['gmaps'])
infra_all_podes.rename(columns={"podes" : "nama"},  inplace=True)
infra_all_gmaps.rename(columns = {"gmaps" : "nama"}, inplace = True)

In [ ]:
#join infra yang ada di podes dengan df yang podes
gmaps_no_dup = gmaps.drop_duplicates(subset = 'nama', keep = 'first')
join_podes = pd.merge(infra_all_podes, podes, on = 'nama')
join_gmaps = pd.merge(infra_all_gmaps, gmaps_no_dup, on = 'nama')

In [ ]:
join_gmaps.shape

(56, 20)

In [ ]:
##save join results both of podes and gmaps to csv file
join_podes.to_csv('Hasil/join_podes.csv', index = False)
join_gmaps.to_csv('Hasil/join_gmaps.csv', index = False)

In [ ]:
#cari berapa ID desa yang cocok
podes_match = pd.read_csv('Hasil/join_podes.csv')
gmaps_match = pd.read_csv('Hasil/join_gmaps.csv')

In [ ]:
if podes_match.iddesa_20201 == gmaps_match.iddesa:
    podes_match['match_id'] == True

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
podes_match['match_id'] = podes_match['iddesa_20201'].isin(gmaps_match['iddesa'])
podes_match['match_longitude'] = podes_match['longitude'].isin(gmaps_match['lon'])
podes_match['match_latitude'] = podes_match['latitude'].isin(gmaps_match['lat'])

In [ ]:
podes_match.to_csv('Hasil/podes_gmaps_match.csv')

In [ ]:
print(sum(podes_match['match_id']))
print(sum(podes_match['match_longitude'])
print(sum(podes_match['match_latitude'])

SyntaxError: invalid syntax (<ipython-input-79-19c33663c666>, line 3)